In [1]:
!pip install -q sentence-transformers


In [8]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
import joblib

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [3]:
df = pd.read_csv('/kaggle/input/diplomacy/train_df.csv')

df.head()

,messages,sender_labels,receiver_labels,speakers,receivers,absolute_message_index,relative_message_index,seasons,years,game_score,game_score_delta,players,game_id,politeness,negative_sentiment,neutral_sentiment,positive_sentiment,vader_score
0,"Tsk tsk, I told you I was right",False,NOANNOTATION,france,germany,492,117,Winter,1909,7,4,"germany,france",6,3.185824,0.0,1.0,0.0,0.0000
1,"Yeah, something tells me that player may be a ...",True,True,england,france,63,8,Spring,1901,3,0,"england,france",1,3.535946,0.0,0.8,0.2,0.9260
2,Regardless of which way it falls in the end.,True,True,england,germany,1477,279,Spring,1906,5,0,"germany,england",1,3.207375,0.0,1.0,0.0,0.0000
3,Austria is going to hedgehog though so I doubt...,False,True,italy,russia,133,9,Spring,1901,3,-1,"italy,russia",7,3.171482,0.0,1.0,0.0,-0.5009
4,I talked to Austria,True,True,germany,england,1363,94,Fall,1902,5,0,"germany,england",2,3.239397,0.0,1.0,0.0,0.0000


In [4]:
train_df = pd.read_csv("/kaggle/input/diplomacy/train_df.csv")
val_df = pd.read_csv("/kaggle/input/diplomacy/val_df.csv")
test_df = pd.read_csv("/kaggle/input/diplomacy/test_df.csv")


In [5]:
X_train_text = train_df['messages'].fillna("")
y_train = train_df['sender_labels'].astype(int)

X_val_text = val_df['messages'].fillna("")
y_val = val_df['sender_labels'].astype(int)

X_test_text = test_df['messages'].fillna("")
y_test = test_df['sender_labels'].astype(int)

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
embedder = SentenceTransformer('sentence-transformers/all-roberta-large-v1', device=device)

X_train = embedder.encode(X_train_text.tolist(), batch_size=32, show_progress_bar=True, device=device)
X_val = embedder.encode(X_val_text.tolist(), batch_size=32, show_progress_bar=True, device=device)
X_test = embedder.encode(X_test_text.tolist(), batch_size=32, show_progress_bar=True, device=device)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.68k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Batches:   0%|          | 0/379 [00:00<?, ?it/s]

Batches:   0%|          | 0/55 [00:00<?, ?it/s]

Batches:   0%|          | 0/109 [00:00<?, ?it/s]

In [9]:
np.save("roberta_X_train.npy", X_train)
np.save("roberta_X_val.npy", X_val)
np.save("roberta_X_test.npy", X_test)


In [10]:
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_val)
print("🔹 Logistic Regression:")
print(classification_report(y_val, y_pred, digits=3))

joblib.dump(logreg, "logreg_model.joblib")


🔹 Logistic Regression:
              precision    recall  f1-score   support

           0      0.000     0.000     0.000        99
           1      0.943     1.000     0.971      1630

    accuracy                          0.943      1729
   macro avg      0.471     0.500     0.485      1729
weighted avg      0.889     0.943     0.915      1729



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


['logreg_model.joblib']

In [11]:
rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_val)
print("🔹 Random Forest:")
print(classification_report(y_val, y_pred, digits=3))

# Save
joblib.dump(rf, "rf_model.joblib")


🔹 Random Forest:
              precision    recall  f1-score   support

           0      0.667     0.020     0.039        99
           1      0.944     0.999     0.971      1630

    accuracy                          0.943      1729
   macro avg      0.805     0.510     0.505      1729
weighted avg      0.928     0.943     0.917      1729



['rf_model.joblib']

In [12]:
svm = SVC(kernel='linear', probability=True)
svm.fit(X_train, y_train)

y_pred = svm.predict(X_val)
print("🔹 SVM:")
print(classification_report(y_val, y_pred, digits=3))

joblib.dump(svm, "svm_model.joblib")


🔹 SVM:
              precision    recall  f1-score   support

           0      0.000     0.000     0.000        99
           1      0.943     1.000     0.971      1630

    accuracy                          0.943      1729
   macro avg      0.471     0.500     0.485      1729
weighted avg      0.889     0.943     0.915      1729



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


['svm_model.joblib']

In [13]:
class MLP(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

X_train_torch = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_torch = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1).to(device)
train_loader = DataLoader(TensorDataset(X_train_torch, y_train_torch), batch_size=64, shuffle=True)

X_val_torch = torch.tensor(X_val, dtype=torch.float32).to(device)
y_val_torch = torch.tensor(y_val.values, dtype=torch.float32).unsqueeze(1).to(device)

mlp_torch = MLP(X_train.shape[1]).to(device)
optimizer = optim.Adam(mlp_torch.parameters(), lr=1e-3)
criterion = nn.BCELoss()

for epoch in range(10):
    mlp_torch.train()
    for xb, yb in train_loader:
        optimizer.zero_grad()
        preds = mlp_torch(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}: Loss = {loss.item():.4f}")


mlp_torch.eval()
with torch.no_grad():
    preds = (mlp_torch(X_val_torch) > 0.5).int().cpu().numpy()
    print("🔹 MLP (PyTorch):")
    print(classification_report(y_val, preds, digits=3))

torch.save(mlp_torch.state_dict(), "mlp_torch_model.pt")


Epoch 1: Loss = 0.0712
Epoch 2: Loss = 0.0277
Epoch 3: Loss = 0.0476
Epoch 4: Loss = 0.0227
Epoch 5: Loss = 0.5172
Epoch 6: Loss = 0.0749
Epoch 7: Loss = 0.1007
Epoch 8: Loss = 0.2370
Epoch 9: Loss = 0.1452
Epoch 10: Loss = 0.0142
🔹 MLP (PyTorch):
              precision    recall  f1-score   support

           0      0.208     0.051     0.081        99
           1      0.945     0.988     0.966      1630

    accuracy                          0.935      1729
   macro avg      0.577     0.519     0.524      1729
weighted avg      0.903     0.935     0.915      1729



In [ ]:
# import gensim.downloader as api

# model = api.load("glove-wiki-gigaword-100")
# embedding = model['hello']  # Get word vector
